# Title
Description

### Step 0: Function that checks if libraries are imported for different environments

In [1]:
# Import the necessary libraries
import importlib  # Allows for runtime importing of modules
import subprocess  # Allows for the execution of bash commands within Python

# Define the function that checks the modules' installation status and import them
def check_installed_and_import(import_dict):
    # Iterate over the dictionary items
    for module, imported_as in import_dict.items():
        try:
            # Attempt to import the module using importlib
            imported_module = importlib.import_module(module)
            # If the import is successful, add the module to the global namespace under the 'imported_as' alias
            globals()[imported_as] = imported_module
            # Print a success message
            print(f"{module} imported as {imported_as}")
        # If the import fails, catch the exception
        except Exception as e:
            # Print a failure message
            print(f"FAILED: {module} imported as {imported_as}. Trying to install...")
            try:
                # Try to install the module using pip via the subprocess module
                base_module = module.split(".")[0]  # Get the base module, in case we're trying to import a submodule
                subprocess.check_call(['pip', 'install', base_module])
                # Try to import the module again
                imported_module = importlib.import_module(module)
                # If the import is successful this time, add the module to the global namespace under the 'imported_as' alias
                globals()[imported_as] = imported_module
                # Print a success message
                print(f"{module} imported as {imported_as}")
            # If the import still fails, catch the exception
            except Exception as e:
                # Print a final failure message along with the exception message
                print(f"FAILED: {module} imported as {imported_as}. Error: {e}")

### Step 1a:  Import necessary libraries

In [2]:
# Call the function with our dictionary to check the modules' installation and import them
check_installed_and_import({
    'graphviz': 'graphviz',
    'IPython.display.Image': 'Img',
    'os': 'os',
    'matplotlib.pyplot': 'plt'
    'numpy': 'np',
    'pydot': 'pydot',
    'tensorflow': 'tf',
    'tensorflow.keras': 'keras',
    'tensorflow.keras.callbacks': 'callbacks',
    'tensorflow.keras.layers': 'layers',
    'tensorflow.keras.models': 'models',
    'tensorflow.keras.optimizers': 'optimizers',
    'tensorflow.keras.mixed_precision.experimental': 'experimental',
    'tensorflow.keras.utils.plot_model', 'plot_model'
    'tensorflow.keras.preprocessing.image': 'image',    
})

os imported as os
numpy imported as np


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

tensorflow imported as tf
tensorflow.keras imported as keras
tensorflow.keras.layers imported as layers
tensorflow.keras.models imported as models
tensorflow.keras.preprocessing.image imported as image
tensorflow.keras.callbacks imported as callbacks
tensorflow.keras.optimizers imported as optimizers


### Step 1b:  Speeding things up.

In [ ]:
# Enable mixed precision training
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

### Step 1c:  variables for flow control

In [ ]:
keras_tuning = True

### Step 2: Data Preprocessing

In [3]:
# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()

# Normalize the data (scale pixel values between 0 and 1)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoded vectors
num_classes = 100
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Data augmentation
data_gen = image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
data_gen.fit(x_train)

169001437/169001437 [==============================] - 2s 0us/step


### Step 3: Build the CNN Architecture

In [4]:
if keras_tuning:
    def build_model(hp):
        model = models.Sequential()
        model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(128, (3, 3), activation='relu'))
        model.add(layers.Flatten())
        model.add(layers.Dense(256, activation='relu'))
        model.add(layers.Dense(num_classes, activation='softmax'))

        # Use hp.Choice to select learning rate
        lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

        model.compile(optimizer=optimizers.Adam(learning_rate=lr),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
        return model
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=5,
        executions_per_trial=3,
        directory='my_dir',
        project_name='Keras_Tuned_CIFAR-100')

    tuner.search(x_train, y_train,
                epochs=5,
                validation_data=(x_test, y_test))

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

tuner.search(x_train, y_train,
            epochs=5,
            validation_data=(x_test, y_test))

model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.4))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))


### Step 4: Compile the Model

In [5]:
opt = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
plot = plot_model(model, show_shapes=True, show_layer_names=True)
plot.savefig("model.png")
Image(filename='model.png')

### Step 5: Training the Model

In [6]:
epochs = 50
batch_size = 64
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(
    data_gen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=len(x_train) / batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping]
)
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Epoch 1/50


2023-08-02 19:37:45.389270: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


781/781 [==============================] - 50s 45ms/step - loss: 4.1079 - accuracy: 0.1020 - val_loss: 3.6259 - val_accuracy: 0.1516
Epoch 2/50
781/781 [==============================] - 33s 42ms/step - loss: 3.2313 - accuracy: 0.2139 - val_loss: 2.8451 - val_accuracy: 0.2929
Epoch 3/50
781/781 [==============================] - 33s 42ms/step - loss: 2.7838 - accuracy: 0.2914 - val_loss: 2.9503 - val_accuracy: 0.2816
Epoch 4/50
781/781 [==============================] - 34s 43ms/step - loss: 2.5086 - accuracy: 0.3483 - val_loss: 2.2082 - val_accuracy: 0.4140
Epoch 5/50
781/781 [==============================] - 33s 42ms/step - loss: 2.3094 - accuracy: 0.3933 - val_loss: 2.1290 - val_accuracy: 0.4408
Epoch 6/50
781/781 [==============================] - 33s 42ms/step - loss: 2.1888 - accuracy: 0.4162 - val_loss: 1.9608 - val_accuracy: 0.4757
Epoch 7/50
781/781 [==============================] - 33s 43ms/step - loss: 2.0813 - accuracy: 0.4404 - val_loss: 1.8370 - val_accuracy: 0.4972
Epo

### Step 6: Evaluate the Model

In [7]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

313/313 [==============================] - 1s 4ms/step - loss: 1.2808 - accuracy: 0.6472
Test Accuracy: 64.72%


### Step 7: Fine-tuning